# Reduce Centre Cell Power

#### **Introduction**

To assess the baseline energy consumption for mobile operator networks running
5G NR SA, this document sets out a scenario for a 5G NR network comprised of a 
19 cell sites arranged in a hexagonal grid with an intersite distance of 1500m.

<!--It isn't but it should be "The scenario is based on the 5G NR SA baseline scenario defined in 3GPP TS 38.101" -->

**Scenario definition** :
[reduce_cell_centre_power.json](KISS/data/input/reduce_centre_cell_power/reduce_centre_cell_power.json).

**Source files** :
[KISS/data/output/reduce_centre_cell_power](KISS/data/output/reduce_centre_cell_power).

In [19]:
from sys import path as sys_path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pathlib import Path
import subprocess
%matplotlib inline

# Check current working directory
path = Path.cwd()
print(f'Current directory path:{path}')

# Get the project directory
kiss_path = path.parents[1]
kiss_path_str = str(kiss_path)
print(f'Project directory path:{kiss_path}')

# Add the project directory to the system path
sys_path.append(kiss_path_str)

Current directory path:/Users/apw804/dev-02/EnergyModels/KISS/data/analysis
Project directory path:/Users/apw804/dev-02/EnergyModels/KISS


### **Process raw files**
Concatenate all the individual tsv files into one DataFrame.
```

In [23]:
# Directory containing the TSV files
tsv_dir = kiss_path / 'data' / 'output' / 'reduce_centre_cell_power' / '2023_03_17' / 'Attempt_003'

# For each tsv file in the directory read the file into a DataFrame and append to a list
df_list = []
for tsv_file in tsv_dir.glob('*.tsv'):
    df_temp = pd.read_csv(tsv_file, sep='\t')
    df_list.append(df_temp)

# Concatenate the list of DataFrames into a single DataFrame
df = pd.concat(df_list)

In [43]:
# What do the first 5 rows of the DataFrame look like?
df.head()

,seed,time,serving_cell_id,serving_cell_sleep_mode,ue_id,distance_to_cell(m),ue_throughput(Mb/s),sc_power(dBm),sc_rsrp(dBm),neighbour1_rsrp(dBm),neighbour2_rsrp(dBm),noise_power(dBm),sinr(dB),cqi,mcs,cell_throughput(Mb/s),cell_power(kW),cell_ee(bits/J),cell_se(bits/Hz)
5,0,1,0,0,21,725.627686,0.4902,43.0,-92.949921,-96.251625,-98.650361,-118,0.681403,2.0,3,20.8086,1.921914,10827.017652,2.08086
5,0,1,0,0,21,725.627686,0.4902,43.0,-92.949921,-96.251625,-98.650361,-118,0.655830,2.0,3,20.8086,1.921914,10827.017652,2.08086
5,0,1,0,0,21,725.627686,0.4902,43.0,-92.949921,-96.251625,-98.650361,-118,0.661392,2.0,3,20.8086,1.921914,10827.017652,2.08086
5,0,1,0,0,21,725.627686,0.4902,43.0,-92.949921,-96.251625,-98.650361,-118,0.703988,2.0,3,20.8086,1.921914,10827.017652,2.08086
5,0,1,0,0,21,725.627686,0.4902,43.0,-92.949921,-96.251625,-98.650361,-118,0.699060,2.0,3,20.8086,1.921914,10827.017652,2.08086


In [45]:
# What does the shape look like?
df.shape

(1080000, 19)

In [46]:
# And the info?
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1080000 entries, 5 to 393
Data columns (total 19 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   seed                     1080000 non-null  int64  
 1   time                     1080000 non-null  int64  
 2   serving_cell_id          1080000 non-null  int64  
 3   serving_cell_sleep_mode  1080000 non-null  int64  
 4   ue_id                    1080000 non-null  int64  
 5   distance_to_cell(m)      1080000 non-null  float64
 6   ue_throughput(Mb/s)      1080000 non-null  float64
 7   sc_power(dBm)            1080000 non-null  float64
 8   sc_rsrp(dBm)             1080000 non-null  float64
 9   neighbour1_rsrp(dBm)     1080000 non-null  float64
 10  neighbour2_rsrp(dBm)     1080000 non-null  float64
 11  noise_power(dBm)         1080000 non-null  int64  
 12  sinr(dB)                 1080000 non-null  float64
 13  cqi                      1080000 non-null  flo

In [48]:
# There is a LOT of noise here. I'll start with the basics for one DataFrame
# and then I'll try to figure out how to do it for all of them.

# Get the data for seed value 0 and sc_power(dBm) value 30.0
df_s000_p30 = df[(df['seed'] == 0) & (df['sc_power(dBm)'] == 30.0)]

# This SHOULD be a dataframe with 16 rows (+1 for the column names)
df_s000_p30.shape

(17, 19)

In [ ]:
# Let's see whatthe whole frame looks like
display(df_s000_p30)

In [42]:
# Okat that's weird. What does the data look like?
df_s000_p30.nunique()

seed                         100
time                           1
serving_cell_id               19
serving_cell_sleep_mode        1
ue_id                        400
distance_to_cell(m)         3152
ue_throughput(Mb/s)          352
sc_power(dBm)                 27
sc_rsrp(dBm)                3594
neighbour1_rsrp(dBm)        3291
neighbour2_rsrp(dBm)        3602
noise_power(dBm)               1
sinr(dB)                   13552
cqi                           16
mcs                           16
cell_throughput(Mb/s)        199
cell_power(kW)                27
cell_ee(bits/J)              199
cell_se(bits/Hz)             199
dtype: int64

In [24]:
# Lets get an overview of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1080000 entries, 0 to 399
Data columns (total 19 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   seed                     1080000 non-null  int64  
 1   time                     1080000 non-null  int64  
 2   serving_cell_id          1080000 non-null  int64  
 3   serving_cell_sleep_mode  1080000 non-null  int64  
 4   ue_id                    1080000 non-null  int64  
 5   distance_to_cell(m)      1080000 non-null  float64
 6   ue_throughput(Mb/s)      1080000 non-null  float64
 7   sc_power(dBm)            1080000 non-null  float64
 8   sc_rsrp(dBm)             1080000 non-null  float64
 9   neighbour1_rsrp(dBm)     1080000 non-null  float64
 10  neighbour2_rsrp(dBm)     1080000 non-null  float64
 11  noise_power(dBm)         1080000 non-null  int64  
 12  sinr(dB)                 1080000 non-null  float64
 13  cqi                      1080000 non-null  flo

In [25]:
# Print the number of unique values in each column
df.nunique()

seed                           100
time                             1
serving_cell_id                 19
serving_cell_sleep_mode          1
ue_id                          400
distance_to_cell(m)          40000
ue_throughput(Mb/s)            635
sc_power(dBm)                   27
sc_rsrp(dBm)                111994
neighbour1_rsrp(dBm)         56589
neighbour2_rsrp(dBm)         84779
noise_power(dBm)                 1
sinr(dB)                   1080000
cqi                             16
mcs                             16
cell_throughput(Mb/s)         9695
cell_power(kW)                  27
cell_ee(bits/J)               9763
cell_se(bits/Hz)              9696
dtype: int64

In [26]:
# Let's get the shape the DataFrame
df.shape

(1080000, 19)

In [28]:
# What do the columns look like?
df.columns

Index(['seed', 'time', 'serving_cell_id', 'serving_cell_sleep_mode', 'ue_id',
       'distance_to_cell(m)', 'ue_throughput(Mb/s)', 'sc_power(dBm)',
       'sc_rsrp(dBm)', 'neighbour1_rsrp(dBm)', 'neighbour2_rsrp(dBm)',
       'noise_power(dBm)', 'sinr(dB)', 'cqi', 'mcs', 'cell_throughput(Mb/s)',
       'cell_power(kW)', 'cell_ee(bits/J)', 'cell_se(bits/Hz)'],
      dtype='object')

In [29]:
# What does the index look like?
df.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            390, 391, 392, 393, 394, 395, 396, 397, 398, 399],
           dtype='int64', length=1080000)

In [ ]:
#  

In [10]:
# Get rows of DataFrame where seed = 1
df_s001 = df[df['seed'] == 1]

df_s001.head()

,index,seed,time,serving_cell_id,serving_cell_sleep_mode,ue_id,distance_to_cell(m),ue_throughput(Mb/s),sc_power(dBm),sc_rsrp(dBm),neighbour1_rsrp(dBm),neighbour2_rsrp(dBm),noise_power(dBm),sinr(dB),cqi,mcs,cell_throughput(Mb/s),cell_power(kW),cell_ee(bits/J),cell_se(bits/Hz)
10800,11607,1,1,0,0,48,332.555178,2.555615,43.0,-79.738874,-101.032757,-103.947262,-118,18.293571,11.0,20,24.480231,1.921914,12737.420617,2.448023
10801,24007,1,1,0,0,48,332.555178,2.555615,43.0,-79.738874,-101.032757,-103.947262,-118,18.304899,11.0,20,24.480231,1.921914,12737.420617,2.448023
10802,52407,1,1,0,0,48,332.555178,2.555615,43.0,-79.738874,-101.032757,-103.947262,-118,18.315826,11.0,20,24.480231,1.921914,12737.420617,2.448023
10803,79607,1,1,0,0,48,332.555178,2.555615,43.0,-79.738874,-101.032757,-103.947262,-118,18.298891,11.0,20,24.480231,1.921914,12737.420617,2.448023
10804,93207,1,1,0,0,48,332.555178,2.555615,43.0,-79.738874,-101.032757,-103.947262,-118,18.303121,11.0,20,24.480231,1.921914,12737.420617,2.448023


In [ ]:
# Get the rows for serving_cell_id = 0
cell_0 = df_copy[df_copy['serving_cell_id'] == 0]

# Get the rows for sc_power = 43dBm
cell_0_p43 = cell_0[cell_0['sc_power(dBm)'] == 43.0]


cell_0_p43.show()


In [ ]:
# Get the mean throughput and standard deviation for each serving_cell_id
cell_throughput_mean_std = df_copy.groupby('serving_cell_id')['cell_throughput(Mb/s)'].agg(['mean', 'std'])
display(cell_throughput_mean_std)

plt.hist()